# Mercari NLP

In [6]:
! pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 15.2 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


In [29]:
import pandas as pd
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/javm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preproccessing

In [30]:
train_df = pd.read_csv("/Users/javm/Desktop/Mercari-Price-Prediction-Project/train.tsv", sep = '\t')

In [31]:
train_df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity
...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...


In [32]:
# Turn 'brand name' to 0s and 1s 

In [33]:
train_df['brand_name'].isna().value_counts()

False    849853
True     632682
Name: brand_name, dtype: int64

In [34]:
# Consider removing category_name values that sum up to less than 50? 150?

In [35]:
train_df['category_name'].value_counts().loc[lambda x: x<200].sum()

30170

In [36]:
print(train_df['item_description'].dtypes)


object


In [37]:
train_df['category_name'] = train_df['category_name'].str.replace('/', ' ')

In [38]:
train_df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men Tops T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics Computers & Tablets Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women Tops & Blouses Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home Home Décor Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women Jewelry Necklaces,NaN,44.0,0,Complete with certificate of authenticity
...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women Dresses Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids Girls 2T-5T Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors Exercise Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home Home Décor Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...


In [39]:
from nltk.corpus import wordnet # imports WordNet
from nltk import pos_tag # nltk's native part of speech tagging

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin

ImportError: dlopen(/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <78036CAA-82A0-3FC2-9DF1-A2F5970A489B> /Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so
  Reason: tried: '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' 

In [27]:
from sklearn.pipeline import Pipeline

ImportError: dlopen(/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <78036CAA-82A0-3FC2-9DF1-A2F5970A489B> /Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so
  Reason: tried: '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' 

In [28]:
from nltk import WordNetLemmatizer # lemmatizer using WordNet

In [25]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/javm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/javm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ImportError: dlopen(/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <78036CAA-82A0-3FC2-9DF1-A2F5970A489B> /Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so
  Reason: tried: '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' 

In [18]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        pass
    
    def fit(self, data, y = 0):
        
        return self
    
    def transform(self, data, y = 0):
        fully_normalized_corpus = data.apply(self.process_doc)
        
        return fully_normalized_corpus
        
    def process_doc(self, doc):

        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None

        doc_norm = [tok.lower() for tok in word_tokenize(doc) if ((tok.isalpha()) & (tok not in stop_words)) ]

        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)

NameError: name 'BaseEstimator' is not defined

In [11]:
proc = TextPreprocessor()

In [12]:
train_df['item_description']

0                                         No description yet
1          This keyboard is in great condition and works ...
2          Adorable top with a hint of lace and a key hol...
3          New with tags. Leather horses. Retail for [rm]...
4                  Complete with certificate of authenticity
                                 ...                        
1482530    Lace, says size small but fits medium perfectl...
1482531     Little mermaid handmade dress never worn size 2t
1482532            Used once or twice, still in great shape.
1482533    There is 2 of each one that you see! So 2 red ...
1482534    New with tag, red with sparkle. Firm price, no...
Name: item_description, Length: 1482535, dtype: object

In [13]:
# Dropped less than 10 rows
train_df.dropna(subset = ['item_description'], inplace = True)

In [14]:
train_df['item_description'].isna().value_counts()

False    1482531
Name: item_description, dtype: int64

In [15]:
# Dropped .4%. Not 40%, but .4% as in .004
train_df.dropna(subset = ['category_name'],inplace = True)

In [16]:
train_df['category_name'].isna().value_counts()

False    1476204
Name: category_name, dtype: int64

In [17]:
df1 = proc.fit_transform(train_df['item_description'])

In [18]:
df2 = proc.fit_transform(train_df['category_name'])

In [19]:
df3 = proc.fit_transform(train_df['name'])

In [20]:
df12 = df1.str.cat(df2, sep = ' ')

In [21]:
df123 = df12.str.cat(df3, sep = ' ')

In [22]:
df123

0          description yet men top mlb cincinnati red t s...
1          keyboard great condition work come port test w...
2          adorable top hint lace key hole back pale pink...
3          new tag leather horse retail rm stand foot hig...
4          complete certificate authenticity woman jewelr...
                                 ...                        
1482530    lace say size small fit medium perfectly never...
1482531    little mermaid handmade dress never wear size ...
1482532    use twice still great shape sport outdoors exe...
1482533    see so red orange big red orange one world mar...
1482534    new tag red sparkle firm price free shipping w...
Name: item_description, Length: 1476204, dtype: object

In [23]:
#total_description_df= pd.Series(["df123"],name="total_description")
total_description_df = df123.to_frame()

In [24]:
total_description_df.rename(columns ={'item_description' : 'total_description'}, inplace = True)

In [25]:
total_description_df

,total_description
0,description yet men top mlb cincinnati red t s...
1,keyboard great condition work come port test w...
2,adorable top hint lace key hole back pale pink...
3,new tag leather horse retail rm stand foot hig...
4,complete certificate authenticity woman jewelr...
...,...
1482530,lace say size small fit medium perfectly never...
1482531,little mermaid handmade dress never wear size ...
1482532,use twice still great shape sport outdoors exe...
1482533,see so red orange big red orange one world mar...


In [27]:
merged_text_df = total_description_df.to_csv(index=False)

## Import Merged Text

In [28]:
df = pd.concat([train_df, total_description_df], axis = 1, join = 'inner')
df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,total_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men Tops T-shirts,NaN,10.0,1,No description yet,description yet men top mlb cincinnati red t s...
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics Computers & Tablets Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,keyboard great condition work come port test w...
2,2,AVA-VIV Blouse,1,Women Tops & Blouses Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,adorable top hint lace key hole back pale pink...
3,3,Leather Horse Statues,1,Home Home Décor Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,new tag leather horse retail rm stand foot hig...
4,4,24K GOLD plated rose,1,Women Jewelry Necklaces,NaN,44.0,0,Complete with certificate of authenticity,complete certificate authenticity woman jewelr...
...,...,...,...,...,...,...,...,...,...
1482530,1482530,Free People Inspired Dress,2,Women Dresses Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",lace say size small fit medium perfectly never...
1482531,1482531,Little mermaid handmade dress,2,Kids Girls 2T-5T Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,little mermaid handmade dress never wear size ...
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors Exercise Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.",use twice still great shape sport outdoors exe...
1482533,1482533,World markets lanterns,3,Home Home Décor Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,see so red orange big red orange one world mar...


In [29]:
ready_df = df.drop(['name', 'category_name', 'item_description'], axis=1)

ready_df

,train_id,item_condition_id,brand_name,price,shipping,total_description
0,0,3,NaN,10.0,1,description yet men top mlb cincinnati red t s...
1,1,3,Razer,52.0,0,keyboard great condition work come port test w...
2,2,1,Target,10.0,1,adorable top hint lace key hole back pale pink...
3,3,1,NaN,35.0,1,new tag leather horse retail rm stand foot hig...
4,4,1,NaN,44.0,0,complete certificate authenticity woman jewelr...
...,...,...,...,...,...,...
1482530,1482530,2,Free People,20.0,1,lace say size small fit medium perfectly never...
1482531,1482531,2,Disney,14.0,0,little mermaid handmade dress never wear size ...
1482532,1482532,2,NaN,12.0,0,use twice still great shape sport outdoors exe...
1482533,1482533,3,NaN,45.0,1,see so red orange big red orange one world mar...


In [30]:
ready_df['brand_name'] = ready_df['brand_name'].fillna(0)

In [31]:
ready_df

,train_id,item_condition_id,brand_name,price,shipping,total_description
0,0,3,0,10.0,1,description yet men top mlb cincinnati red t s...
1,1,3,Razer,52.0,0,keyboard great condition work come port test w...
2,2,1,Target,10.0,1,adorable top hint lace key hole back pale pink...
3,3,1,0,35.0,1,new tag leather horse retail rm stand foot hig...
4,4,1,0,44.0,0,complete certificate authenticity woman jewelr...
...,...,...,...,...,...,...
1482530,1482530,2,Free People,20.0,1,lace say size small fit medium perfectly never...
1482531,1482531,2,Disney,14.0,0,little mermaid handmade dress never wear size ...
1482532,1482532,2,0,12.0,0,use twice still great shape sport outdoors exe...
1482533,1482533,3,0,45.0,1,see so red orange big red orange one world mar...


In [32]:
ready_df['brand_mention'] = np.where(ready_df['brand_name']!= 0, True, False)
ready_df.head()

,train_id,item_condition_id,brand_name,price,shipping,total_description,brand_mention
0,0,3,0,10.0,1,description yet men top mlb cincinnati red t s...,False
1,1,3,Razer,52.0,0,keyboard great condition work come port test w...,True
2,2,1,Target,10.0,1,adorable top hint lace key hole back pale pink...,True
3,3,1,0,35.0,1,new tag leather horse retail rm stand foot hig...,False
4,4,1,0,44.0,0,complete certificate authenticity woman jewelr...,False


In [33]:
one_hot_df = pd.get_dummies(ready_df, prefix="brand_mention", 
                            columns=["brand_mention"], 
                            drop_first=True)

In [34]:
one_hot_df

,train_id,item_condition_id,brand_name,price,shipping,total_description,brand_mention_True
0,0,3,0,10.0,1,description yet men top mlb cincinnati red t s...,0
1,1,3,Razer,52.0,0,keyboard great condition work come port test w...,1
2,2,1,Target,10.0,1,adorable top hint lace key hole back pale pink...,1
3,3,1,0,35.0,1,new tag leather horse retail rm stand foot hig...,0
4,4,1,0,44.0,0,complete certificate authenticity woman jewelr...,0
...,...,...,...,...,...,...,...
1482530,1482530,2,Free People,20.0,1,lace say size small fit medium perfectly never...,1
1482531,1482531,2,Disney,14.0,0,little mermaid handmade dress never wear size ...,1
1482532,1482532,2,0,12.0,0,use twice still great shape sport outdoors exe...,0
1482533,1482533,3,0,45.0,1,see so red orange big red orange one world mar...,0


In [35]:
df = one_hot_df.drop(['brand_name'], axis=1)

In [36]:
# 0 - Did not mention brand name
# 1 - DID mention brand name
df

,train_id,item_condition_id,price,shipping,total_description,brand_mention_True
0,0,3,10.0,1,description yet men top mlb cincinnati red t s...,0
1,1,3,52.0,0,keyboard great condition work come port test w...,1
2,2,1,10.0,1,adorable top hint lace key hole back pale pink...,1
3,3,1,35.0,1,new tag leather horse retail rm stand foot hig...,0
4,4,1,44.0,0,complete certificate authenticity woman jewelr...,0
...,...,...,...,...,...,...
1482530,1482530,2,20.0,1,lace say size small fit medium perfectly never...,1
1482531,1482531,2,14.0,0,little mermaid handmade dress never wear size ...,1
1482532,1482532,2,12.0,0,use twice still great shape sport outdoors exe...,0
1482533,1482533,3,45.0,1,see so red orange big red orange one world mar...,0


## Sample Size

In [37]:
sample_df = df.sample(n=100000)

In [38]:
sample_df.shape

(100000, 6)

In [39]:
sample_df

,train_id,item_condition_id,price,shipping,total_description,brand_mention_True
1461753,1461753,2,10.0,1,never play cheap actual case cd cd hold case e...,0
1168383,1168383,2,36.0,0,item total rm i change total cost item rm rm s...,0
560086,560086,3,12.0,1,removable strap size m item available till jul...,1
1082736,1082736,3,13.0,0,description deep brown combat lace camp boot b...,1
659824,659824,3,7.0,1,large size barely worn kid girl top kid,0
...,...,...,...,...,...,...
155591,155591,1,16.0,0,pack beauty fragrance candle home scent glade ...,0
839989,839989,3,14.0,0,coral top only victoria secret size white cora...,1
30880,30880,1,51.0,0,kendra scott lee iridescent drusy gold earring...,1
1320537,1320537,2,10.0,0,top adorable maroon color cute detail neckline...,1


In [40]:
prc_steps = [('tfid', TfidfVectorizer(min_df = 1, max_df = 1))]
tfid_pipeline = Pipeline(prc_steps)

In [41]:
processed_text = tfid_pipeline.fit_transform(sample_df['total_description'])

In [42]:
processed_text

<100000x18441 sparse matrix of type '<class 'numpy.float64'>'
	with 18441 stored elements in Compressed Sparse Row format>

In [43]:
feat_names = tfid_pipeline['tfid'].get_feature_names()

word_vec = pd.DataFrame(processed_text.toarray(), columns = feat_names)


In [44]:
word_vec

,aaaahhh,aaahh,aab,aad,aadenim,aaliyah,aandf,aape,aardvark,aarm,...,ᴛᴏᴘɪᴄ,ᴡᴀsʜᴇᴅ,ᴡᴏᴜʟᴅ,ℹdetailed,ꮪꭲꮜꭰꮪ,ꮮꭺꮯꭼ,방법,사용하는,혜택,ｆａｑ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
forest_df = pd.concat([sample_df, word_vec], axis =1,join='inner')

In [53]:
forest_df.drop(['total_description'], axis = 1,inplace = True)

In [54]:
forest_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6654 entries, 89294 to 30880
Columns: 18446 entries, train_id to ｆａｑ
dtypes: float64(18442), int64(3), uint8(1)
memory usage: 936.4 MB


In [4]:
from sklearn.model_selection import train_test_split

y = forest_df['price']
X = forest_df.drop(columns = ['price'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

ImportError: dlopen(/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <78036CAA-82A0-3FC2-9DF1-A2F5970A489B> /Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/_iterative.cpython-310-darwin.so
  Reason: tried: '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/lib/python3.10/site-packages/scipy/sparse/linalg/_isolve/../../../../../../liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' (no such file), '/Users/javm/miniforge3/bin/../lib/liblapack.3.dylib' 

In [56]:
from sklearn.ensemble import RandomForestRegressor

In [61]:
rf = RandomForestRegressor(random_state = 42)

In [62]:
rf.fit(X_train,y_train)

RandomForestRegressor(random_state=42)

In [63]:
rf.score(X_train,y_train)

0.8264085314153411

In [64]:
rf.score(X_test,y_test)

-0.606343209095491

 ## Define Sigmoid

## NN

In [91]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
X_train.shape

NameError: name 'X_train' is not defined

In [2]:
# Original OG CNN
model = Sequential()
model.add(Input(shape= (number of dimensions in data)))
model.add(Dense(128, activation='relu'))
# model_reg.add(Dropout(0.2))
# model.add(Dense(128, activation=‘relu’))
# model.add(Dense(64, activation=‘relu’))
# model_reg.add(Dropout(0.2))
# model.add(Dense(32, activation=‘relu’))
model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error',
              optimizer='adam', metrics='accuracy')
model.summary()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (743059382.py, line 3)